In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import multinetx as mnet
import re
from datetime import datetime, timedelta
import pickle
import random
import os
import torch

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/shared/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# --------------------------
# PARAMETERS & THRESHOLDS
# --------------------------
TAU_C = 0.39        # Threshold for content similarity (CSL)
TAU_A = 0.99      # Threshold for affective similarity (ASL)
OMEGA = 1.0        # Uniform interlayer coupling weight
DELTA_T = 3600     # Time window (in seconds) for temporal burst calculation (e.g., 1 hour)

In [2]:
def load_submissions(submissions_file):
    return pd.read_csv(submissions_file, compression="zstd")

def load_comments(comments_file):
    return pd.read_csv(comments_file, compression="zstd")
    
submissions = pd.read_csv('submissions.csv.zst')
submissions['selftext'] = submissions['selftext'].fillna("")
comments = pd.read_csv('comments.csv.zst')

In [3]:
all_authors = list(submissions['author'])
all_authors.extend(list(comments['author']))
all_authors = list(set(all_authors))
print(len(all_authors))

130430


In [4]:
### Build UIL

def get_edges():
    edges = {}

    for comment in comments.itertuples():
        author = comment.author
        parent = comment.parent_id.split("_")[1]
    
        reply_to = None
    
        try:
            reply_to = submissions.loc[submissions['id'] == parent]['author'].values[0]
        except:
            reply_to = comments.loc[comments['id'] == parent]['author'].values[0]
    
        if reply_to:
            edge = (author, reply_to)
            if edge in edges:
                edges[edge] += 1
            else:
                edges[edge] = 1    

    return edges

UIL = nx.DiGraph()
for author in all_authors:
    UIL.add_node(author)

for edge, weight in get_edges().items():
    UIL.add_edge(edge[0], edge[1], weight=weight)

print(f"UIL nodes: {len(UIL.nodes())}")
print(f"UIL edges: {len(UIL.edges())}")
print(f"UIL density: {nx.density(UIL)}")

def average_edge_weight(G):
    """Compute the average edge weight in a NetworkX graph."""
    edge_weights = [data["weight"] for _, _, data in G.edges(data=True)]  # Extract weights
    return sum(edge_weights) / len(edge_weights) if edge_weights else 0 

avg_weight = average_edge_weight(UIL)
print(f"Average UIL Edge Weight: {avg_weight:.4f}")

with open('uil.pkl', 'wb') as f:
    pickle.dump(UIL, f)

UIL nodes: 130430
UIL edges: 439606
UIL density: 2.584115686947797e-05
Average UIL Edge Weight: 1.2130


In [7]:
### Build TDL

def compute_sliding_window_max(timestamps, window=DELTA_T):
    """
    Given a sorted list of timestamps (in seconds), compute the maximum number of events
    occurring within any window of length `window`.
    """
    if len(timestamps) == 0:
        return 0
    max_count = 0
    start = 0
    for end in range(len(timestamps)):
        while timestamps[end] - timestamps[start] > window:
            start += 1
        count = end - start + 1
        if count > max_count:
            max_count = count
    return max_count

def get_edges():
    edges = {}

    for comment in comments.itertuples():
        author = comment.author
        parent = comment.parent_id.split("_")[1]
    
        reply_to = None
    
        try:
            reply_to = submissions.loc[submissions['id'] == parent]['author'].values[0]
        except:
            reply_to = comments.loc[comments['id'] == parent]['author'].values[0]
    
        if reply_to:
            edge = (author, reply_to)
            timestamp = comment.created_utc
            
            if edge in edges:
                edges[edge].append(timestamp)
            else:
                edges[edge] = [timestamp]   

    return edges

TDL = nx.DiGraph()
for author in all_authors:
    TDL.add_node(author)

for edge, times in get_edges().items():
    times_sorted = sorted(times)
    burst_weight = compute_sliding_window_max(times_sorted, window=DELTA_T)
    TDL.add_edge(edge[0], edge[1], weight=burst_weight)

print(f"TDL nodes: {len(TDL.nodes())}")
print(f"TDL edges: {len(TDL.edges())}")
print(f"TDL density: {nx.density(TDL)}")

def average_edge_weight(G):
    """Compute the average edge weight in a NetworkX graph."""
    edge_weights = [data["weight"] for _, _, data in G.edges(data=True)]  # Extract weights
    return sum(edge_weights) / len(edge_weights) if edge_weights else 0 

avg_weight = average_edge_weight(TDL)
print(f"Average TDL Edge Weight: {avg_weight:.4f}")

with open('tdl.pkl', 'wb') as f:
    pickle.dump(TDL, f)

TDL nodes: 130430
TDL edges: 439606
TDL density: 2.584115686947797e-05
Average TDL Edge Weight: 1.1241


In [2]:
# --------------------------
# CLEANING FUNCTION
# --------------------------
def clean(text):
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

# --------------------------
# LOAD DATA
# --------------------------
def load_submissions(submissions_file):
    return pd.read_csv(submissions_file, compression="zstd")

def load_comments(comments_file):
    return pd.read_csv(comments_file, compression="zstd")

submissions = pd.read_csv("submissions.csv.zst")
comments = pd.read_csv("comments.csv.zst")


# CLS/ASL Preprocessing
# --------------------------
# AGGREGATE TEXTS PER AUTHOR
# --------------------------
def aggregate_texts(df, text_fields):
    """Concatenate selected fields into a single string per row."""
    return df[text_fields].fillna("").agg(" ".join, axis=1)

submissions["full_text"] = aggregate_texts(submissions, ["title", "selftext"])
comments["body"] = comments["body"].fillna("")
submissions["selftext"] = submissions["selftext"].fillna("")

contents_per_author = {}

# Process submissions
for submission in submissions.itertuples():
    author = submission.author
    full_text = clean(f"{submission.title} {submission.selftext}")
    contents_per_author.setdefault(author, []).append(full_text)

# Process comments
for comment in comments.itertuples():
    author = comment.author
    body = clean(comment.body)
    contents_per_author.setdefault(author, []).append(body)

# Merge all texts for each author
for author, contents in contents_per_author.items():
    contents_per_author[author] = " ".join(contents)

authors = list(contents_per_author.keys())
contents = list(contents_per_author.values())

os.makedirs('cslasl-pre', exist_ok=True)
with open('cslasl-pre/authors.pkl', 'wb') as f:
    pickle.dump(authors, f)
with open('cslasl-pre/contents.pkl', 'wb') as f:
    pickle.dump(contents, f)

In [ ]:
# Get CSL edges and with tau_c at 0.3, then compute estimated tau_c

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", trust_remote_code=True, device="cuda:1")
embeddings = model.encode(contents, normalize=True)
with open('cslasl-pre/csl_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [8]:
from tqdm import tqdm

# Compute cosine similarity in **batches** to prevent MemoryError
num_users = len(authors)
batch_size = 1000  # Adjust based on memory constraints

# Initialize tqdm progress bar
total_batches = (num_users // batch_size) ** 2  # Total iterations (i, j) pairs
progress_bar = tqdm(total=total_batches, desc="Computing Similarities", unit="batch")

for i in range(0, num_users, batch_size):
    edges = {}
    end_i = min(i + batch_size, num_users)
    
    for j in range(i, num_users, batch_size):
        end_j = min(j + batch_size, num_users)
        
        # Compute batch-wise similarity
        similarity_batch = np.dot(embeddings[i:end_i], embeddings[j:end_j].T)
        
        # Extract upper triangle indices to avoid duplicate calculations
        if i == j:
            upper_tri_indices = np.triu_indices(end_i - i, k=1)
            similarity_values = similarity_batch[upper_tri_indices]
        else:
            similarity_values = similarity_batch.flatten()

        # Store only meaningful similarities
        for x, sim in enumerate(similarity_values):
            if sim > 0.3:  # Adjust threshold as needed
                author_1 = authors[i + x // (end_j - j)]
                author_2 = authors[j + x % (end_j - j)]
                edges[(author_1, author_2)] = sim

        # Update progress bar
        progress_bar.update(1)
                
    # Save edges to a file
    os.makedirs('cslasl-pre/edges', exist_ok=True)
    with open(f"cslasl-pre/edges/edges_{i}.pkl", "wb") as f:
        pickle.dump(edges, f)
    del edges

# Close progress bar
progress_bar.close()

Computing Similarities:  51%|████████████████▎               | 8646/16900 [2:08:26<2:02:37,  1.12batch/s]


In [4]:
import math
import glob

def custom_round(num):
    """
    Ceil if the third digit after the decimal is >= 5, otherwise floor.
    Always returns a number with two decimal places, but prevents rounding to 1.00.
    """
    # Convert number to string for precise control
    num_str = f"{num:.10f}"  # Ensure precision to check the third decimal
    integer_part, decimal_part = num_str.split('.')

    # Extract digits
    second_digit = int(decimal_part[1]) if len(decimal_part) > 1 else 0
    third_digit = int(decimal_part[2]) if len(decimal_part) > 2 else 0

    # Compute the base rounded value
    base_value = math.floor(num * 100) / 100  # Default floor rounding

    # Only increase within the 2 decimal boundary
    if third_digit >= 5:
        adjusted_value = base_value + 0.01  # Move up without crossing integer boundary
        return min(adjusted_value, math.floor(num))  # Prevent rounding to 1.00

    return base_value  # Default floored value


# Compute empirical threshold
edges_files = glob.glob(os.path.join("cslasl-pre/edges", "*.pkl"))

similarity_values = []

for edges_file in edges_files:
    with open(edges_file, "rb") as f:
        current = pickle.load(f)    
    similarity_values.extend(current.values())
    del current

    # Optional: Manually trigger garbage collection
    import gc
    gc.collect()

TAU_C = custom_round(np.percentile(similarity_values, 90))  # 90th percentile
print("Estimated tau_c (90th percentile):", TAU_C)

del similarity_values

all_edges = {}
for edges_file in edges_files:
    with open(edges_file, "rb") as f:
        current = pickle.load(f)
    for key, value in current.items():
        if float(value) > TAU_C:
            all_edges[key] = float(value)
    del current

with open(f"cslasl-pre/edges/all_edges.pkl", "wb") as f:
    pickle.dump(all_edges, f)

Estimated tau_c (90th percentile): 0.5


In [2]:
### Build CSL
with open("cslasl-pre/edges/all_edges.pkl", "rb") as f:
    edges = pickle.load(f)

with open("cslasl-pre/authors.pkl", "rb") as f:
    authors = pickle.load(f)

CSL = nx.Graph()
for author in authors:
    CSL.add_node(author)

del authors

keys = list(edges.keys())
for key in keys:
    CSL.add_edge(key[0], key[1], weight=float(edges[key]))
    del edges[key]

del edges
del keys

print(f"CSL nodes: {len(CSL.nodes())}")
print(f"CSL edges: {len(CSL.edges())}")
print(f"CSL density: {nx.density(CSL)}")

def average_edge_weight(G):
    """Compute the average edge weight in a NetworkX graph."""
    edge_weights = [data["weight"] for _, _, data in G.edges(data=True)]  # Extract weights
    return sum(edge_weights) / len(edge_weights) if edge_weights else 0 

avg_weight = average_edge_weight(CSL)
print(f"Average CSL Edge Weight: {avg_weight:.4f}")

with open('csl.pkl', 'wb') as f:
    pickle.dump(CSL, f)

CSL nodes: 130430
CSL edges: 186477809
CSL density: 0.02192327818567331
Average CSL Edge Weight: 0.5557


In [6]:
# Get ASL edges and with tau_a at 0.5, then compute estimated tau_a

with open("cslasl-pre/authors.pkl", "rb") as f:
    authors = pickle.load(f)
with open("cslasl-pre/contents.pkl", "rb") as f:
    contents = pickle.load(f)

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment").to("cuda:1")

def get_sentiment_vector(text):
    """Compute sentiment vector from RoBERTa model."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to("cuda:1")
    with torch.no_grad():
        output = model(**inputs).logits  # Get logits
    return output.squeeze().cpu().numpy()

authors_sentiments = {}
for i in range(0, len(authors)):
    authors_sentiments[authors[i]] = get_sentiment_vector(contents[i])

with open("cslasl-pre/edges/authors_sentiments.pkl", 'wb') as f:
    pickle.dump(authors_sentiments, f)

del authors
del contents

In [7]:
# Convert sentiment vectors to NumPy
sentiment_matrix = np.array(list(authors_sentiments.values()))
norms = np.linalg.norm(sentiment_matrix, axis=1, keepdims=True)
valid_norms = norms.flatten() > 0  # Mask for non-zero norms
normalized_matrix = np.zeros_like(sentiment_matrix)
normalized_matrix[valid_norms] = sentiment_matrix[valid_norms] / norms[valid_norms]

with open("cslasl-pre/edges/normalized_matrix.pkl", 'wb') as f:
    pickle.dump(normalized_matrix, f)

In [9]:
from tqdm import tqdm

with open("cslasl-pre/authors.pkl", "rb") as f:
    authors = pickle.load(f)

# Compute cosine similarity in **batches** to prevent MemoryError
num_users = len(authors)
batch_size = 200  # Adjust based on memory constraints

# Initialize tqdm progress bar
total_batches = (num_users // batch_size) ** 2  # Total iterations (i, j) pairs
progress_bar = tqdm(total=total_batches, desc="Computing Similarities", unit="batch")

# Compute cosine similarity in **batches** for sentiment analysis
for i in range(0, num_users, batch_size):
    edges_sentiment = {}
    end_i = min(i + batch_size, num_users)
    
    for j in range(i, num_users, batch_size):
        end_j = min(j + batch_size, num_users)
        
        # Compute batch-wise similarity
        similarity_batch = np.dot(normalized_matrix[i:end_i], normalized_matrix[j:end_j].T)
        
        # Extract upper triangle indices to avoid duplicate calculations
        if i == j:
            upper_tri_indices = np.triu_indices(end_i - i, k=1)
            similarity_values = similarity_batch[upper_tri_indices]
        else:
            similarity_values = similarity_batch.flatten()

        # Store only meaningful similarities
        for x, sim in enumerate(similarity_values):
            if sim > 0.5:  # Adjust threshold as needed
                author_1 = authors[i + x // (end_j - j)]
                author_2 = authors[j + x % (end_j - j)]
                edges_sentiment[(author_1, author_2)] = sim

        # Update progress bar
        progress_bar.update(1)
                
    # Save edges to a file
    os.makedirs('cslasl-pre/edges', exist_ok=True)
    with open(f"cslasl-pre/edges/edges_sentiments_{i}.pkl", "wb") as f:
        pickle.dump(edges_sentiment, f)
    del edges_sentiment

Computing Similarities:  50%|██████████████▌              | 213531/425104 [5:27:19<2:45:41, 21.28batch/s]

In [ ]:
import math
import glob
from tdigest import TDigest
from tqdm import tqdm

digest = TDigest()

def custom_round(num):
    """
    Ceil if the fourth digit after the decimal is >= 5, otherwise floor.
    Always returns a number with three decimal places.
    """
    # Convert number to string for precise control
    num_str = f"{num:.10f}"  # Ensure precision to check the fourth decimal
    integer_part, decimal_part = num_str.split('.')

    # Extract digits
    third_digit = int(decimal_part[2]) if len(decimal_part) > 2 else 0
    fourth_digit = int(decimal_part[3]) if len(decimal_part) > 3 else 0  # Fourth digit

    # Compute the base rounded value (3 decimal places)
    base_value = math.floor(num * 1000) / 1000  # Default floor rounding

    # Only increase within the 3 decimal boundary
    if fourth_digit >= 5:
        adjusted_value = base_value + 0.001  # Move up without crossing integer boundary
        return min(adjusted_value, math.floor(num * 100) / 100)  # Prevent rounding to unintended values

    return base_value  # Default floored value


# Compute empirical threshold
edges_files = glob.glob(os.path.join("cslasl-pre/edges", "edges_sentiments_*.pkl"))
progress_bar = tqdm(total=len(edges_files), desc="Computing TAU_A", unit="batch")


for edges_file in edges_files:
    with open(edges_file, "rb") as f:
        current = pickle.load(f)
    # Update the digest with each similarity value
    for value in current.values():
        digest.update(value)
    del current
    progress_bar.update(1)

estimated_90th = digest.percentile(90)
TAU_A = custom_round(estimated_90th)
print("Estimated tau_a (90th percentile):", TAU_A)
del digest

progress_bar = tqdm(total=len(edges_files), desc="Generating all edges")
current_edges = {}
for i in range(0, len(edges_files)):
    with open(edges_files[i], "rb") as f:
        current = pickle.load(f)
        current_edges = {key: f_val for key, value in current.items() if (f_val := float(value)) > TAU_A}
    del current
    with open(f"cslasl-pre/edges/all_edges_sentiment_{i}.pkl", "wb") as f:
        pickle.dump(current_edges, f)
    del current_edges
    progress_bar.update(1)

In [5]:
import math
import glob
from tdigest import TDigest
from tqdm import tqdm

TAU_A = 0.998
edges_files = glob.glob(os.path.join("cslasl-pre/edges", "edges_sentiments_*.pkl"))
progress_bar = tqdm(total=len(edges_files), desc="Generating all edges")
current_edges = {}
for i in range(0, len(edges_files)):
    with open(edges_files[i], "rb") as f:
        current = pickle.load(f)
        current_edges = {key: f_val for key, value in current.items() if (f_val := float(value)) > TAU_A}
    del current
    with open(f"cslasl-pre/edges/all_edges_sentiment_{i}.pkl", "wb") as f:
        pickle.dump(current_edges, f)
    del current_edges
    progress_bar.update(1)

Generating all edges: 100%|█████████████████████████████████████| 653/653 [1:52:45<00:00,  4.37s/it]

In [ ]:
from tqdm import tqdm
import glob

### Build ASL
with open("cslasl-pre/authors.pkl", "rb") as f:
    authors = pickle.load(f)

ASL = nx.Graph()
progress_bar = tqdm(total=len(authors), desc="Adding authors")
for author in authors:
    ASL.add_node(author)
    progress_bar.update(1)
print(f"ASL nodes: {len(ASL.nodes())}")
del authors


edges_files = glob.glob(os.path.join("cslasl-pre/edges", "all_edges_sentiment_*.pkl"))
progress_bar = tqdm(total=len(edges_files), desc="Adding edges from files")
for edges_file in edges_files:
    with open(edges_file, "rb") as f:
        edges = pickle.load(f)
        for edge, weight in edges.items():
            ASL.add_edge(edge[0], edge[1], weight=weight)
        del edges
    progress_bar.update(1)

with open('asl.pkl', 'wb') as f:
    pickle.dump(ASL, f)
    
print(f"ASL edges: {len(ASL.edges())}")
print(f"ASL density: {nx.density(ASL)}")

def average_edge_weight(G):
    """Compute the average edge weight in a NetworkX graph."""
    edge_weights = [data["weight"] for _, _, data in G.edges(data=True)]  # Extract weights
    return sum(edge_weights) / len(edge_weights) if edge_weights else 0 

avg_weight = average_edge_weight(ASL)
print(f"Average ASL Edge Weight: {avg_weight:.4f}")

Adding authors:  93%|███████████████████████████████▌  | 121117/130430 [00:00<00:00, 1211162.49it/s]

ASL nodes: 130430



Adding authors: 100%|██████████████████████████████████| 130430/130430 [00:00<00:00, 1061088.10it/s]

Adding edges from files:  27%|█████████▊                          | 178/653 [08:08<38:41,  4.89s/it]

In [2]:
import pickle
import glob
import os
from tqdm import tqdm

# --------------------------
# Write Authors to Disk
# --------------------------
with open("cslasl-pre/authors.pkl", "rb") as f:
    authors = pickle.load(f)

# Write authors to a text file (one per line)
with open("cslasl-pre/edges/authors.txt", "w", encoding="utf-8") as fout:
    for author in authors:
        fout.write(f"{author}\n")

num_nodes = len(authors)
print(f"Total number of nodes (authors): {num_nodes}")
del authors

# --------------------------
# Write Edges Directly to Disk
# --------------------------
edges_files = glob.glob(os.path.join("cslasl-pre/edges", "all_edges_sentiment_*.pkl"))
edges_output_file = "cslasl-pre/edges/all_edges_asl.txt"

# Open the output file in write mode; each line will contain: source target weight
with open(edges_output_file, "w", encoding="utf-8") as fout:
    progress_bar = tqdm(total=len(edges_files), desc="Processing edge files", unit="file")
    for edges_file in edges_files:
        with open(edges_file, "rb") as f:
            edges = pickle.load(f)
        for edge, weight in edges.items():
            fout.write(f"{edge[0]} {edge[1]} {weight}\n")
        del edges
        progress_bar.update(1)
    progress_bar.close()

# --------------------------
# Compute Graph Metrics by Streaming the Edge File
# --------------------------
num_edges = 0
weight_sum = 0.0

with open(edges_output_file, "r", encoding="utf-8") as fin:
    for line in fin:
        parts = line.strip().split()
        if len(parts) == 3:
            try:
                w = float(parts[2])
                weight_sum += w
                num_edges += 1
            except ValueError:
                continue

print(f"Total number of edges: {num_edges}")

# Compute average edge weight
avg_edge_weight = weight_sum / num_edges if num_edges > 0 else 0
print(f"Average edge weight: {avg_edge_weight:.4f}")

# Compute graph density for an undirected graph:
# Density = 2 * E / (N*(N-1))
if num_nodes > 1:
    density = 2 * num_edges / (num_nodes * (num_nodes - 1))
else:
    density = 0
print(f"Graph density: {density:.6f}")

Total number of nodes (authors): 130430


Processing edge files: 100%|████████████████████████████████████| 653/653 [26:14<00:00,  2.41s/file]


Total number of edges: 757759444
Average edge weight: 0.9992
Graph density: 0.089086


In [ ]:
import networkx as nx

def edge_generator(edge_file):
    """Yield one edge at a time from the edge file."""
    with open(edge_file, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 3:
                source, target, weight = parts
                if float(weight) > 0.999:
                    yield source, target, {"weight": float(weight)}

# Create an empty graph
G = nx.Graph()

# Add nodes from the authors.txt file
with open("cslasl-pre/edges/authors.txt", "r", encoding="utf-8") as f:
    for line in f:
        node = line.strip()
        if node:
            G.add_node(node)
print(f"Added {G.number_of_nodes()} nodes.")

# Add edges by streaming from the edge file
G.add_edges_from(edge_generator("cslasl-pre/edges/all_edges_asl.txt"))
print(f"Added {G.number_of_edges()} edges.")

# Save the graph to disk in gpickle format
nx.write_gpickle(G, "asl.pkl")

Added 130430 nodes.


In [134]:
with open('uil.pkl', 'rb') as f:
    UIL = pickle.load(f)
with open('csl.pkl', 'rb') as f:
    CSL = pickle.load(f)
with open('tdl.pkl', 'rb') as f:
    TDL = pickle.load(f)
with open('asl.pkl', 'rb') as f:
    ASL = pickle.load(f)

layers = {
    "UIL": UIL,
    "CSL": CSL,
    "TDL": TDL,
    "ASL": ASL
}

In [136]:
multilayer = mnet.MultilayerGraph()

# Add nodes from UIL (the same for all layers).
for layer_name, G in layers.items():
    for node in G.nodes():
        multilayer.add_node((node, layer_name))

# Add intralayer edges.
for layer_name, G in layers.items():
    for u, v, data in G.edges(data=True):
        multilayer.add_edge((u, layer_name), (v, layer_name), weight=data.get('weight', 1))

# Add interlayer edges.
all_users = set()
for G in layers.values():
    all_users.update(G.nodes())
layer_names = list(layers.keys())
for user in all_users:
    for i in range(len(layer_names)):
        for j in range(i+1, len(layer_names)):
            multilayer.add_edge((user, layer_names[i]), (user, layer_names[j]), weight=OMEGA)

del layers

interlayer_edges = [e for e in multilayer.edges() if e[0][1] != e[1][1]]
num_interlayer_edges = len(interlayer_edges)

print("Multilayer network constructed with {} interlayer edges.".format(len(interlayer_edges)))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [137]:
# --------------------------
# SAVE THE MULTILAYER NETWORK
# --------------------------
with open('network.pkl', 'wb') as f:
    pickle.dump(multilayer, f)
print("Multilayer network saved to 'multilayer_network.pkl'")

Multilayer network saved to 'multilayer_network.pkl'
